In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install opencv-python

In [22]:
# Traigo las librerias del google para realizar la conexion con el drive
from google.colab import files
from google.colab import drive

In [23]:
drive.mount("/content/drive") # Se realiza la peticion para conexion a la cuenta de google drive

Mounted at /content/drive


In [24]:
pathOrigen = r"/content/drive/MyDrive/Diversa/imagenes".replace("\\","/")
path_sr_images_folder = r"/content/drive/MyDrive/Diversa/ResultadoImagenesObject".replace("\\","/")

In [25]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow  # Importar cv2_imshow para mostrar imágenes en Colab
import pandas as pd
import os

# Cargar el modelo YOLOv8
model = YOLO('yolov8n.pt')

In [34]:
def yoloFct(imagePath):
    """
    Esta función ayuda a detectar la mayor cantidad de objetos en una imagen.
    """
    # Verifica si la imagen fue cargada correctamente
    image = cv2.imread(imagePath)
    if image is None:
        print(f"Error: No se pudo cargar la imagen en la ruta {imagePath}")
        return [], None

    # Realizar la detección
    results = model(image)

    detection_results = []

    # Procesar resultados
    for result in results:
        for box in result.boxes:
            cls = int(box.cls)  # Clase del objeto detectado
            conf = float(box.conf)  # Confianza de la detección

            # Convertir los tensores a enteros
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Asume que box.xyxy es un tensor de tamaño [1, 4]
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Obtener el nombre de la clase
            class_name = model.names[cls]

            # Formatear la etiqueta
            label = f"{class_name} {conf:.2f}"

            # Calcular la posición del texto
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            y = max(y1, h + 10)

            # Dibujar un rectángulo de fondo para la etiqueta
            cv2.rectangle(image, (x1, y - h - 5), (x1 + w, y + 5), (0, 255, 0), -1)

            # Poner el texto sobre el rectángulo de fondo
            cv2.putText(image, label, (x1, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

            detection_results.append({
                'image_id': os.path.splitext(os.path.basename(imagePath))[0],
                'class': class_name,
                'confidence': conf
            })

    return detection_results, image

In [35]:
# Obtener la lista de imágenes
input_imagenes_list = os.listdir(pathOrigen)

# Crear la carpeta de resultados si no existe
if not os.path.exists(path_sr_images_folder):
    os.makedirs(path_sr_images_folder)

# Inicializar una lista para almacenar todos los resultados de detección
all_detection_results = []

for image_name in input_imagenes_list:
    # Procesar cada imagen
    image_path = os.path.join(pathOrigen, image_name)
    detection_results, processed_image = yoloFct(image_path)

    # Guardar los resultados de detección en la lista
    all_detection_results.extend(detection_results)

    # Guardar la imagen procesada con detecciones
    if processed_image is not None:
        result_image_name = image_name.split(".")[0] + "_detected.jpg"
        final_path = os.path.join(path_sr_images_folder, result_image_name)
        cv2.imwrite(final_path, processed_image)

df = pd.DataFrame(all_detection_results)

# Guardar el DataFrame como un archivo CSV localmente
output_csv = 'detection_results.csv'
df.to_csv(output_csv, index=False)

print(f"Resultados guardados en {output_csv}")


0: 640x640 3 persons, 3 bicycles, 2 cars, 1 bus, 1 truck, 270.2ms
Speed: 10.1ms preprocess, 270.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 3 cars, 1 motorcycle, 238.8ms
Speed: 8.1ms preprocess, 238.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1 car, 1 motorcycle, 1 truck, 1 umbrella, 233.1ms
Speed: 7.2ms preprocess, 233.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 motorcycle, 228.0ms
Speed: 6.1ms preprocess, 228.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Resultados guardados en detection_results.csv
